### Predição de propriedades moleculares com aprendizado de máquina

O projeto busca aplicar técnicas de aprendizado de máquina para otimização de processos laboratoriais. Para isso, contamos com o carregamento de dados experimentais do dataset [QM9](http://quantum-machine.org/datasets/), o qual disponibiliza a informação da energia **homo - lumo**, o **momento de dipolo** e o número de átomos de mais de 130000 moléculas orgânicas, indexadas pelos respectivos SMILES, as quais são formadas pelos átomos de carbono, oxigênio, fluor e nitrogênio.

In [1]:
#library import

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns 
from rdkit import Chem

#local module import
import sys
import os
sys.path.append(os.path.abspath("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular")) 
from src.modeling_utils import DataIngestion
from src.model_train2 import evaluate_models_cv_regression

In [2]:
# data input

df_mol = pd.read_csv("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular/data/origin_dataset.csv", header=0)

print("Número de moléculas:", df_mol.shape[0])
print(50*"-")
print("Colunas:")
df_mol.head(0)


Número de moléculas: 133885
--------------------------------------------------
Colunas:


,smiles,n_atoms,homo_lumo_nm,dipole_moment_debye


In [3]:
# Checking if there is NaN values

df_mol.isna().sum()

smiles                 0
n_atoms                0
homo_lumo_nm           0
dipole_moment_debye    0
dtype: int64

In [17]:
# Dropping homo_lumo columns - because is a highly correlated with dipole columns and its a possible target in the others preditions

df_mol = df_mol.drop(["homo_lumo_nm"], axis=1)

#### **Tratamento e ingestão de dados**

Nesta etapa aplicamos uma linha de processamento contida na classe **DataIngestion()** para gerar um novo conjunto de dados, os processos são apresentados abaixo:

- **drop_duplicated()** : Método utilizado para retirar moléculas duplicadas do conjunto de dados original. De acordo com as referências do projeto, existe a possibilidade de a mesma molécula ser escrita com SMILES diferentes, como 'C=CCC' e 'CCC=C', apesar de representarem a mesma estrutura, são escritos de maneira diferente devido a simetria.

- **get_features()** : Método utilizado para gerar características das moléculas e adicionar ao conjunto original. As novas características serão geradas através do resultado de todos os descritores presentes na biblioteca RDKit (biblioteca voltada a química computacional).

- **get_best_features()**: Método que seleciona as características com melhor perfil para serem usadas em dados de predição. Buscamos selecionar as features de acordo com o valor de correlação e da variância, dado que colunas com grandes correlações ou pequena variância foram excluídas.

Por fim, os métodos podem ser executados separadamente ou, ao utilizar o método **execute_pipeline()**, temos a execução em sequência de todos os métodos, assim como em um **pipeline**.

In [18]:
data_ingestion = DataIngestion(data=df_mol.head(1000), name_column_smiles="smiles")
data_ingestion.execute_pipeline()

In [19]:
df_mol_new = pd.read_csv("/home/edmurcn/Documentos/MeusProjetos/Predicao_Molecular/data/data_best_features.csv", header=0)
print("Número de moléculas:", df_mol_new.shape[0])
print(100*"-")
print("Novas Colunas:", df_mol_new.shape[1])
df_mol_new.head(0)

Número de moléculas: 1000
----------------------------------------------------------------------------------------------------
Novas Colunas: 86


,smiles,n_atoms,dipole_moment_debye,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,MolWt,BCUT2D_MWHI,BCUT2D_MRHI,BalabanJ,...,fr_NH0,fr_NH1,fr_NH2,fr_allylic_oxid,fr_amide,fr_bicyclic,fr_ether,fr_ketone,fr_nitrile,fr_piperdine


#### **Redução de dimensionalidade**

Nesta etapa testamos algums métodos para reduzir ainda mais a dimensão dos dados. Serão aplicados métodos como **PCA()**, **LLA()**... 

A componente principal é um vetor unitário compostos por contribuições de cada eixo, com esses vetor formamos hiperplanos e projetamos os dados nestes hiperplanos para obter o máximo de vâriancia explicada no mínimo de dimensões possíveis.

## Realizar a exploração dos dados